In [1]:
################################################################################################
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank)와 BM25 측정하는 예시임
# => bm25 설치 : !pip install rank_bm25
#
# 1. 검색모델 로딩
# 2. json QuA파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
# 3. 정답리스트에 대해 임베딩 벡터 생성 후 FAISS에 인덱싱함.
# 4. 쿼리를 list로 만들고, 검색 후 예측된 결과를 list로 만듬
# 5. 정답리스트와 예측리스트를 가지고 MRR을 구함
# 6. contexts df 를 BM25 인덱싱 하고 , questions df 를 쿼리로 입력하여 BM25 스코어 구함
################################################################################################
import faiss
import numpy as np
import pandas as pd
import time
import json
import os
from tqdm.notebook import tqdm

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything, mlogging

logger = mlogging(loggername="MRR-BM25", logfilename="../../../log/MRR-BM25")
device = GPU_info()

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500            # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5               # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
use_cross_encoder = True   # cross_encoder 사용할지 유.무 (true=사용함, false=사용안함)
use_bm25 = True            # BM25 출력 할지=True. 안할지=False
embedding_paragraph_avg = False # True = 문장 임베딩 구할때 여러문장 벡터 평균으로 구함(느림), Fals=문장 전체를 하나의 벡터로 생성(빠름)
faiss_index_method = 0      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)
#------------------------------------------------------------------------------------

# param 인자 범위 체크
if faiss_index_method > 1 or faiss_index_method < 0:
    raise ValueError(f"faiss_index_method = {faiss_index_method} is not bad!!")

seed_everything(seed)

logfilepath:../../../log/MRR-BM25_2023-02-08.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [2]:
#-------------------------------------------------------------------------------------
# 1.sentence bert 모델 로딩
#-------------------------------------------------------------------------------------
from myutils import bi_encoder
from sentence_transformers.cross_encoder import CrossEncoder
bi_encoder_path = "bongsoo/albert-small-kor-sbert-v1.1" # kpf-sbert-v1.1" # klue-sbert-v1 # albert-small-kor-sbert-v1.1 # kpf-sbert-v1.1
pooling_mode = 'mean' # mean # cls

bi_encoder = bi_encoder(model_path=bi_encoder_path, max_seq_len=256, do_lower_case=True, pooling_mode=pooling_mode)
bi_encoder.to(device)

# cross-encoder 모델 로딩
if use_cross_encoder == True:
    cross_encoder_path = "bongsoo/albert-small-kor-cross-encoder-v1"# klue-cross-encoder-v1 # albert-small-kor-cross-encoder-v1 # kpf_cross-encoder-v1
    cross_encoder = CrossEncoder(cross_encoder_path, max_length=512, device=device)


In [3]:
#-------------------------------------------------------------------------------------------------------
# 2. KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json, read_aihub_qua_json

# aihub QuA 파일을 불러옴.
jsonfile = './data/VL_text_entailment.json' # VL_unanswerable.json # VL_text_entailment.json # VL_span_inference.json # KorQuAD_v1.0_dev.json
contexts, questions, answers, contextids, qcontextids = read_aihub_qua_json(jsonfile) # read_aihub_qua_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [4]:
df_contexts.head()

,context,contextid
0,두올산업은 캐나다 법인 온코퀘스트의 난소암 면역 항암 신약 ‘오레고보맙’에 대한 미...,10001
1,법무부가 '이태원발 코로나19 감염사태'와 관련해 외국인들에 대한 관리 강화에 나선...,10002
2,유안타증권이 14일 미국주식 투자자들을 대상으로 글로벌 금융정보회사 레피니티브(Re...,10003
3,인천 남동구에서 '이태원 클럽'을 방문한 학원강사로부터 코로나19 감염된 고3 모자...,10004
4,헬스에어테크놀로지코리아(HATK)의 독일 공기청정기 브랜드 나노드론이 ‘공기의 세계...,10005


In [5]:
df_questions.head()

,question,answer,contextid
0,오레고보맙의 임상이 전 세계 환자들을 상대로 실시돼,Yes,10001
1,전 세계 환자들을 상대로 오레고보맙의 임상이 시행돼,Yes,10001
2,이태원 클럽에 간 외국인들의 코로나 자진 검사를 법무부가 유도하고 있어,Yes,10002
3,법무부는 이태원 클럽과 관련된 외국인들에게 코로나 자진 검사를 하도록 유도했어,Yes,10002
4,유안타증권은 미국주식 유망 종목 자동검색 시스템을 오픈했어,Yes,10003


In [6]:
#---------------------------------------------------------
# 3. 임베딩 벡터 생성 후 FAISS에 인덱싱함.
#---------------------------------------------------------
from myutils import embed_text_avg

#=============================================================================
# 문장을 .(마침표)로 여러 문장으로 나누고 나눈문장을 1개씩 임베딩 구한후 계수만큼 나워서 평균 임베딩 구하기
# => 속도 느림, 최대 15개만 함.(CPU 환경에서는 좋음)
#=============================================================================
def paragraph_index(paragraph):
    avg_paragraph_vec = np.zeros((1,768))
    sent_count = 0
    
    # ** kss로 분할할때 히브리어: מר, 기타 이상한 특수문자 있으면 에러남. 
    # 따라서 여기서는 그냥 . 기준으로 문장을 나누고 평균을 구함
    # 하나의 문장을 읽어와서 .기준으로 나눈다.
    sentences = [sentence for sentence in paragraph.split('. ') if sentence != '' and len(sentence) > 20]
    
    for sent in sentences:
        # 문장으로 나누고, 해당 vector들의 평균을 구함.
        #avg_paragraph_vec += embed_text([sent])
        avg_paragraph_vec += embed_text(model=bi_encoder, contexts=[sent], return_tensor=False)
        sent_count += 1
  
        # 최대 15개 문장만 처리함 
        if sent_count >= 15:
            break
         
    '''
    # kss로 분할할때 줄바꿈 있으면, 파싱하는데 에러남.따라서 "\n"는 제거함
    paragraph = paragraph.replace("\n","")
    
    print("==Start paragraph_index==")
    print(paragraph)
    for sent in kss.split_sentences(paragraph):
        # 문장으로 나누고, 해당 vector들의 평균을 구함.
        avg_paragraph_vec += embed_text([sent])
        sent_count += 1
        
        # 최대 10개 문장만 처리함 
        if sent_count >= 10:
            break
    '''
 
    # 0으로 나누면 배열이 nan(not a number)가 되어 버리므로, 반드시 0>큰지 확인해야 함
    if sent_count > 0:
        avg_paragraph_vec /= sent_count
    
    return avg_paragraph_vec.ravel(order='C') # 1차원 배열로 변경
   
#=============================================================================
# 임베딩 벡터 생성하여 FAISS에 인덱싱하고 ID와 매핑 처리하는 함수
# => IN : contextdf
# => OUT : FASSI index
#=============================================================================
def embeddingforfaiss(df, use_paragraph_avg):
           
    # embedding 생성(인코딩)
    start = time.time()
    paragraphs = df.context.to_list()
    print(f'*임베딩 할 context 계수: {len(paragraphs)}') 
    
    # paragraph_avg == True 이면 평균 문자 임베딩 벡터 구함.
    if use_paragraph_avg == True:
        print(f'*----평균 문장 임베딩 벡터 구하기----')
        embeddings = [embed_text_avg(model=bi_encoder, contexts=paragraph) for paragraph in tqdm(paragraphs)]
    else:
        embeddings = bi_encoder.encode(paragraphs, show_progress_bar=True, convert_to_tensor=False)
       
    embeddings = np.array([embedding for embedding in embeddings]).astype("float32")#float32 로 embeddings 타입 변경
    #print(type(embeddings)) #print(embeddings.shape) #print(embeddings[0])    
    
    # instance index 생성
    # => IndexFlatL2 : Euclidean Distance 측정함
    # => IndexFlatIP : cosine 유사도 측정함 => faiss.normalize_L2(embeddings) 호출해줘야 함.
    if faiss_index_method == 0:      # 0=Cosine Similarity 적용(IndexFlatIP 사용), 
        index = faiss.IndexFlatIP(embeddings.shape[1])
        faiss.normalize_L2(embeddings)# *cosine유사도 구할때는 반드시 normalize 처리함.
    elif faiss_index_method == 1:    # 1=Euclidean Distance 적용(IndexFlatL2 사용)
        index = faiss.IndexFlatL2(embeddings.shape[1])
    
    # id를 매핑 시켜줌 => 이때 idtype은 반드시 int64 type이어야 함.
    index = faiss.IndexIDMap2(index)

    index.add_with_ids(embeddings, df.contextid.values)

    print(f'*임베딩 시간 : {time.time()-start:.4f}')
    
    return index

#df 임베딩 구하고 faiss에 추가함.
index = embeddingforfaiss(df_contexts, use_paragraph_avg = embedding_paragraph_avg)

*임베딩 할 context 계수: 3001


Batches:   0%|          | 0/94 [00:00<?, ?it/s]

*임베딩 시간 : 7.3745


In [7]:
#----------------------------------------------------------------
# 4. 쿼리를 list로 만들고, 검색 후 예측된 결과를 list로 만듬
#----------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import df_sampling

# Query를 list로 만들고 -> query 인코딩후->검색 결과 출력

if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_query = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_sampling(df=df_questions, num=query_num, seed=seed)
    user_query = df_questions['question'].values.tolist()

print(f'Query-----------------------------------------------------')
print(user_query[0:3])

vector = bi_encoder.encode(user_query)
if faiss_index_method == 0:
    faiss.normalize_L2(vector)              # *cosine유사도 구할때는 반드시 normalize 처리함.
    
distance, idx = index.search(np.array(vector).astype("float32"), k=search_k)

# 예측검색결과를 리스트로 만듬.
bi_predictions_list = []

for i, query in enumerate(tqdm(user_query)):
    bi_predictions_list.append(idx[i].tolist())
    
print(f'----------------------------------------------------------')
print(f'bi-encoder 예측:{len(bi_predictions_list)}')
print(bi_predictions_list[0:3])

# cross-encoder 사용인 경우에
# - 한번더 검색된 결과에 {쿼리, 문장} 쌍으로 만들어서 cross-encoder로 스코어 출력함.
if use_cross_encoder == True:
    # {쿼리, 문장} 쌍 만듬.
    #count = 0
    cross_predictions_list = []
    for i, predicts in enumerate(tqdm(bi_predictions_list)):
        sentence_combinations = []
        query = user_query[i]
        #count += 1
             
        for predict in predicts:  
             sentence_combinations.append([query, df_contexts[df_contexts.contextid == predict]['context'].values.tolist()[0]])
                     
        # cross-enocoder 돌려서 출력된 score 추가함.
        cross_scores = cross_encoder.predict(sentence_combinations)+1
        #print(f'*cross_scores:{cross_scores}')
        
        dec_cross_idx = reversed(np.argsort(cross_scores))
        tmp_list = []
        for idx in dec_cross_idx:
            #print(f'idx:{idx}-predicts:{predicts[idx]}')
            tmp_list.append(predicts[idx])
         
        cross_predictions_list.append(tmp_list)   
    
    print(f'----------------------------------------------------------')
    print(f'cross-encoder 예측:{len(cross_predictions_list)}')
    print(f'{cross_predictions_list[0:3]}')


Query-----------------------------------------------------
['막걸리 제조 회사의 지분을 플루토가 사들였어', '버핏을 속인 독일 기업에 관한조사가 시행되고 있어', '중견기업계는 국민의 요청을 인지해야 한다고 21대 국회에게 발언했어']


  0%|          | 0/500 [00:00<?, ?it/s]

----------------------------------------------------------
bi-encoder 예측:500
[[10511, 10655, 10457, 11455, 12432], [10622, 11146, 11455, 10063, 10743], [10214, 11146, 12635, 11398, 11354]]


  0%|          | 0/500 [00:00<?, ?it/s]

----------------------------------------------------------
cross-encoder 예측:500
[[10655, 12432, 10511, 11455, 10457], [11455, 10063, 10743, 11146, 10622], [10214, 11398, 11146, 12635, 11354]]


In [8]:
#--------------------------------------------------------------------------------------------------
# 5. MRR을 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

logger.info(f'--------------------------------------------------------------------------')
logger.info('json_file:{}'.format(jsonfile))
logger.info(f'faiss 인덱싱 방식: {faiss_index_method}(0=코사인, 1=유클리드)')
logger.info(f'문장 평균 임베딩 유.무: {embedding_paragraph_avg}(true=적용됨, false=적용안됨)')
logger.info('search_k:{}/query_num:{}'.format(search_k, query_num))

# MRR를 구함
if use_cross_encoder == True:
    predictions_list = bi_predictions_list
    bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

    # BI-MRR 출력
    logger.info(f'--------------------------------------------------------------------------')
    logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
    logger.info('*BI-MRR:{:.4f}'.format(bi_score))
    logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
    predictions_list = cross_predictions_list
    cross_ranks, cross_score = mean_reciprocal_rank(ground_truths_list, predictions_list)
    logger.info(f'---------------------------------------------------------------------------')
    logger.info('*CROSS-ENCODER:{}'.format(cross_encoder_path))
    logger.info('*CROSS-MRR:{:.4f}'.format(cross_score))
    logger.info(f'*Ranks({len(cross_ranks)}):{cross_ranks[0:20]}')
    
    # 검색 못한 계슈
    print(f'---------------------------------------------------------------------------')
    zero_count = 0
    for item in bi_ranks:
        if item == 0:
            zero_count += 1
    
    logger.info('*검색실패계수: {}/{}({:.2f}%)'.format(zero_count, len(bi_ranks), (zero_count/len(bi_ranks))*100))
    print(f'---------------------------------------------------------------------------')
    
    logger.info(f'\nBI -> CROSS---------------------------------------------------------------------------')
    df_scores = pd.DataFrame((zip(bi_ranks, cross_ranks)), columns = ['bi-rank','cross-rank'])
    logger.info(df_scores.head(20))

else:
    predictions_list = bi_predictions_list

    bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

    # BI-MRR 출력
    logger.info(f'----------------------------------------------------------------------------')
    logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
    logger.info('*BI-MRR:{:.4f}'.format(bi_score))
    logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
    # 검색 못한 계슈
    logger.info(f'---------------------------------------------------------------------------')
    zero_count = 0
    for item in bi_ranks:
        if item == 0:
            zero_count += 1
    
    logger.info('*검색실패계수: {}/{}({:.2f}%)'.format(zero_count, len(bi_ranks), (zero_count/len(bi_ranks))*100))
    logger.info(f'---------------------------------------------------------------------------')

2023-02-08 15:14:07,780 - MRR-BM25 - INFO - --------------------------------------------------------------------------
2023-02-08 15:14:07,783 - MRR-BM25 - INFO - json_file:./data/VL_text_entailment.json
2023-02-08 15:14:07,784 - MRR-BM25 - INFO - faiss 인덱싱 방식: 0(0=코사인, 1=유클리드)
2023-02-08 15:14:07,785 - MRR-BM25 - INFO - 문장 평균 임베딩 유.무: False(true=적용됨, false=적용안됨)
2023-02-08 15:14:07,786 - MRR-BM25 - INFO - search_k:5/query_num:500
2023-02-08 15:14:07,788 - MRR-BM25 - INFO - --------------------------------------------------------------------------
2023-02-08 15:14:07,789 - MRR-BM25 - INFO - *BI-ENCODER:bongsoo/albert-small-kor-sbert-v1.1
2023-02-08 15:14:07,790 - MRR-BM25 - INFO - *BI-MRR:0.1288
2023-02-08 15:14:07,791 - MRR-BM25 - INFO - *Ranks(500):[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3333333333333333, 0]
2023-02-08 15:14:07,792 - MRR-BM25 - INFO - ---------------------------------------------------------------------------
2023-02-08 15:14:07,793 - MRR-BM25 - INFO

---------------------------------------------------------------------------
---------------------------------------------------------------------------


In [9]:
#------------------------------------------------------------------------------------------------
# 6. BM25 계산
# => korquad_v1.0 말뭉치를 가지고 BM25 계산하는 함수
# => for문을 2번 돌면서 BM25 2번 계산함.
#    - 처음에는(0) 해당 쿼리와 contexts에 대해 mecab 적용 후 BM25 스코어 계산, 
#    - 2번째는 mecab 적용하지 않고 계산
#------------------------------------------------------------------------------------------------
if use_bm25 == True:

    import konlpy
    from konlpy.tag import Mecab
    from rank_bm25 import BM25Okapi
    from tqdm.notebook import tqdm

    def BM25tokenizer(sent):
      return sent.split(" ")

    # 입력된 contexts를 mecab을 이용하여 형태소 추출 후 " " 붙여서 형태소 문장을 만듬.
    # Mecab 선언
    mecab = Mecab()

    for count in range(2):
        mecab_str = ''
        # 0이면(처음엔) mecab 적용함=> tokeniaer 후 인덱싱
        if count == 0:
            mecab_str = "(mecab 적용)"
            mecab_contexts=[]
            for context in tqdm(contexts):
                temp = mecab.morphs(context)   # ['세계', '배달', '피자', '리더', '도미노피자','가'..] 식으로 temp 리스트가 생성됨
                sentence = " ".join(temp)      # 위 temp 리스트를 공백을 넣어서 한문장으로 합침 ['세계 배달 피자 리더 도미노피자 가 ...]
                mecab_contexts.append(sentence)

            print(f'*contexts_len:{len(mecab_contexts)}')  
            print(f'{mecab_contexts[0]}')

            # tokeniaer 후 인덱싱
            tokenized_corpus = [BM25tokenizer(doc) for doc in mecab_contexts]
        else:
            # 2번째는 그냥 인덱싱
            tokenized_corpus = [BM25tokenizer(doc) for doc in contexts]
            
        bm25 = BM25Okapi(tokenized_corpus)

        #print(f'bm25.doc_len:{bm25.doc_len}')
        #print(f'type(bm25.doc_freqs):{type(bm25.doc_freqs)}')
        
        bm5_predictions_list = []
     
        # 쿼리 후 get_scores 를 이용하여, scores를 구함.
        for idx, query in enumerate(user_query):
            
            # 처음에는 mecab적용해서 query문 전처리 함.
            if count == 0:
                tempq = mecab.morphs(query) 
                query = " ".join(tempq)
                
            # 쿼리에 따른 스코어 구함    
            tokenized_query = BM25tokenizer(query)
            doc_scores = bm25.get_scores(tokenized_query)

            # 정렬후 최대 스코어 search_k 만큼만 출력함
            top_lists = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)[:search_k]
            bm5_predictions_list.append([index + contextids[0] for index, score in top_lists])

        # 정답, 여기서는 contextid를 리스트로 만듬.
        ground_truths_list = df_questions['contextid'].values.tolist()

        # MPR 계산
        predictions_list = bm5_predictions_list  # 예측 결과 리스트
        bm25_ranks, bm25_score = mean_reciprocal_rank(ground_truths_list, predictions_list)

         # BM25-MRR 출력
        logger.info(f'--------------------------------------------------------------------------')
        logger.info('*BM25-MRR{}:{:.4f}'.format(mecab_str, bm25_score))
        logger.info(f'*Ranks({len(bm25_ranks)}):{bm25_ranks[0:20]}')
        # 검색 못한 계슈
        #logger.info(f'---------------------------------------------------------------------------')
        zero_count = 0
        for item in bm25_ranks:
            if item == 0:
                zero_count += 1

        logger.info('*BM25{} 검색실패계수: {}/{}({:.2f}%)'.format(mecab_str, zero_count, len(bm25_ranks), (zero_count/len(bm25_ranks))*100))
        logger.info(f'---------------------------------------------------------------------------')

  0%|          | 0/3001 [00:00<?, ?it/s]

*contexts_len:3001
두 올 산업 은 캐나다 법인 온 코 퀘스트 의 난소암 면역 항암 신약 ‘ 오레 고보 맙 ’ 에 대한 미국 FDA ( 식품 의 약국 ) 임상 3 상 이 순항 중 이 라고 14 일 밝혔 다 . 온 코 퀘스트 관계자 는 " 2019 년 11 월 4 일 미국 FDA 에 제출 한 IND ( 임상시험 계획 ) 에 대한 지연 통보 를 받 지 않 아 , 12 월 4 일 부터 임상 3 상 이 개시 됐 다 " 며 " 올해 23 월 글로벌 임상시험 수탁 기관 ( CRO ) 업체 IQVIA 와 서비스 계약 을 체결 하 고 , 본격 적 인 환자 모집 을 진행 하 고 있 다 " 고 말 했 다 . 이어 " 현재 북미 , 남미 , 유럽 , 아시아 국가 에 139 곳 사이트 를 통해 총 602 명 의 환자 등록 을 위해 각 사이트 별 환자 모집 을 하 고 있 다 " 며 " 미국 부인 종양학 연구회 ( GOG Foundation ) 와 임상 3 상 관련 서비스 계약 을 체결 하 여 본격 적 인 임상 준비 중 이 다 " 고 설명 했 다 . 온 코 퀘스트 는 현재 코로나 19 영향 으로 일부 지역 의 병원 및 임상 센터 와 의 일정 조율 중 이 다 . 오 는 7 - 8 월 1 차 환자 등록 이 완료 되 고 , 본격 적 인 임상 시험 에 들어갈 것 으로 예상 하 고 있 다 . 이 관계자 는 " 난소암 신규 환자 대상 외 에 도 재발 환자 대상 으로 도 하반기 에 글로벌 제약사 GSK 의 난소암 치료 제 제 줄라 와 병 용 투여 치료 에 대한 임상 을 진행할 예정 " 이 라며 " 췌장암 에 대한 임상 또한 임상 1 / 2 상 을 미국 과 중국 환자 대상 으로 동시 에 진행 할 계획 이 다 " 고 전했 다 . 이어 " 두 올 산업 은 양수 한 무형 자산 파이프라인 의 임상 프로그램 이 순차 적 으로 진행 되 는 데 문제 가 없 도록 충분 한 자금 을 확보 했 다 " 고 덧붙였 다 .


2023-02-08 15:14:21,910 - MRR-BM25 - INFO - --------------------------------------------------------------------------
2023-02-08 15:14:21,912 - MRR-BM25 - INFO - *BM25-MRR(mecab 적용):0.8519
2023-02-08 15:14:21,913 - MRR-BM25 - INFO - *Ranks(500):[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0]
2023-02-08 15:14:21,914 - MRR-BM25 - INFO - *BM25(mecab 적용) 검색실패계수: 28/500(5.60%)
2023-02-08 15:14:21,915 - MRR-BM25 - INFO - ---------------------------------------------------------------------------
2023-02-08 15:14:25,981 - MRR-BM25 - INFO - --------------------------------------------------------------------------
2023-02-08 15:14:25,983 - MRR-BM25 - INFO - *BM25-MRR:0.5067
2023-02-08 15:14:25,984 - MRR-BM25 - INFO - *Ranks(500):[1.0, 1.0, 1.0, 1.0, 0, 1.0, 0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0, 1.0, 0.3333333333333333, 0, 0.3333333333333333, 0, 0.3333333333333333]
2023-02-08 15:14:25,985 - MRR-BM25 - INFO - *BM25 검색실패계수: 200/500(40.00%)
2023-02